In [1]:
import pandas as pd

import psycopg2 as pg

from sqlalchemy import create_engine

## Conectando com o banco de dados

In [2]:
# Configurações da conexão
usuario = 'postgres'
senha = '123,258D'
host = 'localhost'
porta = '5432'
banco_de_dados = 'my_test'

# URL de conexão
url = f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco_de_dados}"

# Criar o engine
engine = create_engine(url)

# Teste a conexão
try:
    connection = engine.connect()
    print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")
finally:
    connection.close()


Conexão bem-sucedida!


## Inserindo arquivo csv no banco de dados conectado

In [3]:
arquivo = '/home/ferdanielinux/desktop-linux/scripts-python/proj-aries/Dados/CNES/tbEstabelecimento202406.csv'

In [ ]:
df = pd.read_csv(arquivo, sep=';', encoding = 'latin-1')

In [5]:
print("Número de linhas: ", df.shape[0])
print("Número de colunas: ", df.shape[1])

Número de linhas:  535727
Número de colunas:  55


In [6]:
# Separando apenas os atributos que serão utilizados no banco de dados
colunas = df.columns

colunas

Index(['CO_UNIDADE', 'CO_CNES', 'NU_CNPJ_MANTENEDORA', 'TP_PFPJ', 'NIVEL_DEP',
       'NO_RAZAO_SOCIAL', 'NO_FANTASIA', 'NO_LOGRADOURO', 'NU_ENDERECO',
       'NO_COMPLEMENTO', 'NO_BAIRRO', 'CO_CEP', 'CO_REGIAO_SAUDE',
       'CO_MICRO_REGIAO', 'CO_DISTRITO_SANITARIO',
       'CO_DISTRITO_ADMINISTRATIVO', 'NU_TELEFONE', 'NU_FAX', 'NO_EMAIL',
       'NU_CPF', 'NU_CNPJ', 'CO_ATIVIDADE', 'CO_CLIENTELA', 'NU_ALVARA',
       'DT_EXPEDICAO', 'TP_ORGAO_EXPEDIDOR', 'DT_VAL_LIC_SANI', 'TP_LIC_SANI',
       'TP_UNIDADE', 'CO_TURNO_ATENDIMENTO', 'CO_ESTADO_GESTOR',
       'CO_MUNICIPIO_GESTOR', 'TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')',
       'CO_USUARIO', 'CO_CPFDIRETORCLN', 'REG_DIRETORCLN',
       'ST_ADESAO_FILANTROP', 'CO_MOTIVO_DESAB', 'NO_URL', 'NU_LATITUDE',
       'NU_LONGITUDE', 'TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')', 'NO_USUARIO_GEO',
       'CO_NATUREZA_JUR', 'TP_ESTAB_SEMPRE_ABERTO',
       'ST_GERACREDITO_GERENTE_SGIF', 'ST_CONEXAO_INTERNET', 'CO_TIPO_UNIDADE',
       'NO_FANTASIA_ABREV',

Atributos de interesse:
CO_UNIDADE, CO_CNES, NU_CNPJ_MANTENEDORA, NO_RAZAO_SOCIAL, NO_FANTASIA, NO_LOGRADOURO, NU_ENDERECO, NO_COMPLEMENTO, NO_BAIRRO, CO_CEP, CO_REGIAO_SAUDE, NU_TELEFONE, NO_EMAIL e DT_EXPEDICAO

In [10]:
df2 = df.filter(items = ['CO_UNIDADE', 'CO_CNES', 'NU_CNPJ_MANTENEDORA', 'NO_RAZAO_SOCIAL', 'NO_FANTASIA', 'NO_LOGRADOURO', 'NU_ENDERECO',
                         'NO_COMPLEMENTO', 'NO_BAIRRO', 'CO_CEP', 'CO_REGIAO_SAUDE', 'NU_TELEFONE', 'NO_EMAIL', 'DT_EXPEDICAO'])

df2.head(10)

,CO_UNIDADE,CO_CNES,NU_CNPJ_MANTENEDORA,NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_COMPLEMENTO,NO_BAIRRO,CO_CEP,CO_REGIAO_SAUDE,NU_TELEFONE,NO_EMAIL,DT_EXPEDICAO
0,2609602569302,2569302,1.040418e+13,PREFEITURA MUNICIPAL DE OLINDA,USF BULTRINS MONTE II,RUA PREFEITO MANOEL REGUEIRA,540,NaN,BULTRINS,53320460,001,(81)34930626,NaN,NaN
1,2609602571943,2571943,1.040418e+13,PREFEITURA MUNICIPAL DE OLINDA,UNIDADE MOVEL,RUA DO SOL,311,NaN,CARMO,53120010,001,(81)34294465,NaN,NaN
2,2609602344637,2344637,1.040418e+13,PREFEITURA MUNICIPAL DE OLINDA,USF ALTO DA MINA,RUA AVENCA,49,NaN,ALTO DA MINA,53250441,001,(81)33051133,NaN,NaN
3,2609602344696,2344696,1.040418e+13,PREFEITURA MUNICIPAL DE OLINDA,USF ILHA DE SANTANA I E II,RUA DA INTEGRACAO,S/N,NaN,JARDIM ATLANTICO,53060001,001,(81)34324703,NaN,NaN
4,3112002142295,2142295,NaN,FUNDACAO COMUNITARIA DE SAUDE DE CANDEIAS,HOSPITAL CARLOS CHAGAS,AVENIDA PEDRO VIEIRA DE AZEVEDO,687,NaN,CENTRO,37280000,15,035-3833.-1285,hospitalcarloschagas@yahoo.com.br,28-set-2017 00:00:00
5,3112602121506,2121506,NaN,LABORATORIO CENTRAL DE CAPINOPOLIS LTDA,LABORATORIO CENTRAL,AV 99,613,NaN,CENTRO,38360000,026,(34)32631058,NaN,28-mai-2002 00:00:00
6,3112902759861,2759861,NaN,LABORATORIO SANTA HELENA LTDA,LABORATORIO SANTA HELENA,AV MANOEL FRANCISCO DE FREITAS,57,NaN,CENTRO,36925000,024,(31)38735235,laboratoriostahelena@ig.com.br,NaN
7,2917702533324,2533324,1.398832e+13,MUNICIPIO DE JAGUARARI,POSTO DE SAUDE DE ABOBORA,POVOADO DE ABOBORA,S/N,TERREO,ZONA RURAL,48960000,28,(07)45414196,semusjaguar@bol.com.br,NaN
8,2923604029542,4029542,1.367549e+13,MUNICIPIO DE PARAMIRIM,UNIDADE DO PSF DE CANABRAVINHA,DISTRITO DE CANABRAVINHA,S/N,NaN,ZONA RURAL,46190000,023,(77)34712412,smsaudeparamirim@bol.com.br,NaN
9,4218506433839,6433839,NaN,CONSULTORIO ODONTOLOGICO JOAO R MARANHA,CONSULTORIO ODONTOLOGICO JOAO R MARANHA,RUA JOSE BONIFACIO,10,1 ANDAR SALA 11,CENTRO,89650000,7ª,(49)35370376,jmmaranha@brturbo.com.br,23-jun-2014 00:00:00


Acertando os tipos dos dados

In [5]:
df.to_sql('teste2', engine, if_exists='replace', index=False)

5

## Lendo arquivos do banco de dados

In [6]:
df2 = pd.read_sql('teste2', engine)

df2

,UF_ZI,ANO_CMPT,MES_CMPT,ESPEC,CGC_HOSP,N_AIH,IDENT,CEP,MUNIC_RES,NASC,...,DIAGSEC9,TPDISEC1,TPDISEC2,TPDISEC3,TPDISEC4,TPDISEC5,TPDISEC6,TPDISEC7,TPDISEC8,TPDISEC9
0,350000,2015,5,3,NaN,3515108800808,1,17560000,355660,19570120,...,NaN,0,0,0,0,0,0,0,0,0
1,350000,2015,5,3,NaN,3515108800820,1,17601180,355500,19350425,...,NaN,1,1,1,0,0,0,0,0,0
2,350000,2015,5,3,NaN,3515108800863,1,17501010,352900,19590331,...,NaN,0,0,0,0,0,0,0,0,0
3,350000,2015,5,3,NaN,3515108800874,1,17505500,352900,19280229,...,NaN,1,1,0,0,0,0,0,0,0
4,350000,2015,5,1,NaN,3515108789786,1,19970000,353530,19750701,...,NaN,0,0,0,0,0,0,0,0,0
